# Full Stage HAADF Mosaic Acquisition
This notebook guides you through acquiring and stitching HAADF images covering the entire TEM stage.

In [1]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import math
import os
from datetime import datetime
from pathlib import Path
import json

In [2]:
from stemOrchestrator.logging_config import setup_logging

data_folder = "."
out_path = data_folder
setup_logging(out_path=out_path)

## Acquisition and Processing Functions
All functions for acquisition, stitching, and visualization are defined below.

In [ ]:
def collect_full_stage_haadf_mosaic(tf_acquisition, config):
    """
    Collect HAADF images covering the entire stage range and stitch them together

    Args:
        tf_acquisition: TFacquisition object
        config: Configuration dictionary with imaging parameters
    """

    # Image acquisition parameters
    exposure = config.get("haadf_exposure", 200e-9)  # seconds
    resolution = config.get("haadf_resolution", 512)
    out_path = Path(config.get("out_path", "."))
    overlap = config.get("overlap", 0.1)  # 10% overlap between adjacent images

    # Create output directory with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = out_path / f"full_stage_haadf_mosaic_{timestamp}"
    output_dir.mkdir(exist_ok=True)

    print("Analyzing stage limits and calculating grid...")

    # Get microscope and stage information
    microscope = tf_acquisition.microscope
    stage = microscope.specimen.stage

    # Get stage limits
    x_limits = stage.get_axis_limits(axis="x")
    y_limits = stage.get_axis_limits(axis="y")

    print(f"Stage X limits: {x_limits.min * 1e6:.1f} to {x_limits.max * 1e6:.1f} μm")
    print(f"Stage Y limits: {y_limits.min * 1e6:.1f} to {y_limits.max * 1e6:.1f} μm")

    # Get field of view
    fov = microscope.optics.scan_field_of_view
    print(f"Field of view: {fov * 1e6:.2f} μm")

    # Calculate step size with overlap
    step_size = fov * (1 - overlap)
    print(f"Step size (with {overlap * 100:.1f}% overlap): {step_size * 1e6:.2f} μm")

    # Calculate stage ranges
    stage_range_x = x_limits.max - x_limits.min
    stage_range_y = y_limits.max - y_limits.min

    print(
        f"Total stage range: X={stage_range_x * 1e6:.1f} μm, Y={stage_range_y * 1e6:.1f} μm"
    )

    # Calculate number of images needed in each direction
    ImagesX = math.ceil(stage_range_x / step_size) + 1
    ImagesY = math.ceil(stage_range_y / step_size) + 1

    print(
        f"Calculated grid size: {ImagesX} × {ImagesY} = {ImagesX * ImagesY} total images"
    )
    print(
        f"Estimated acquisition time: {(ImagesX * ImagesY * exposure * resolution * resolution / 60):.1f} minutes"
    )

    # Ask for confirmation
    response = input(f"This will acquire {ImagesX * ImagesY} images. Continue? (y/n): ")
    if response.lower() != "y":
        print("Acquisition cancelled.")
        return None

    # Store initial stage position
    InitialStagePosition = stage.position
    print(
        f"Initial stage position: X={InitialStagePosition.x * 1e6:.2f}μm, Y={InitialStagePosition.y * 1e6:.2f}μm"
    )

    # Calculate starting position (bottom-left corner of stage)
    start_x = x_limits.min
    start_y = y_limits.min

    # Lists to store images and metadata
    images = []
    image_data = []
    positions = []
    filenames = []

    # Create metadata file
    metadata_file = output_dir / "acquisition_metadata.txt"
    with open(metadata_file, "w") as f:
        f.write(f"Full Stage HAADF Mosaic Acquisition\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"Grid size: {ImagesX} × {ImagesY}\n")
        f.write(f"Total images: {ImagesX * ImagesY}\n")
        f.write(f"Field of view: {fov * 1e6:.2f} μm\n")
        f.write(f"Step size: {step_size * 1e6:.2f} μm\n")
        f.write(f"Overlap: {overlap * 100:.1f}%\n")
        f.write(f"Exposure: {exposure * 1e6:.1f} μs\n")
        f.write(f"Resolution: {resolution} × {resolution}\n")
        f.write(
            f"Stage X range: {x_limits.min * 1e6:.1f} to {x_limits.max * 1e6:.1f} μm\n"
        )
        f.write(
            f"Stage Y range: {y_limits.min * 1e6:.1f} to {y_limits.max * 1e6:.1f} μm\n"
        )
        f.write(
            f"Initial position: X={InitialStagePosition.x * 1e6:.2f}, Y={InitialStagePosition.y * 1e6:.2f} μm\n\n"
        )
        f.write("Image positions:\n")
        f.write("Index\tGrid_X\tGrid_Y\tStage_X(μm)\tStage_Y(μm)\tFilename\n")

    try:
        image_index = 0

        # Loop through all positions
        for grid_y in range(ImagesY):
            for grid_x in range(ImagesX):
                image_index += 1
                print(
                    f"Acquiring image {image_index}/{ImagesX * ImagesY} at grid position ({grid_x}, {grid_y})"
                )

                # Calculate actual stage position
                stage_x = start_x + grid_x * step_size
                stage_y = start_y + grid_y * step_size

                # Ensure we don't exceed stage limits
                stage_x = max(x_limits.min, min(x_limits.max, stage_x))
                stage_y = max(y_limits.min, min(y_limits.max, stage_y))

                # Move stage to new position
                from autoscript_tem_microscope_client.structures import StagePosition

                target_position = StagePosition(
                    x=stage_x, y=stage_y, z=InitialStagePosition.z
                )

                print(f"  Moving to: X={stage_x * 1e6:.2f}μm, Y={stage_y * 1e6:.2f}μm")
                stage.absolute_move(target_position)

                # Verify actual position
                actual_position = stage.position
                positions.append((actual_position.x, actual_position.y))

                # Acquire HAADF image
                try:
                    haadf_image, haadf_data, haadf_filename, pixel_size = (
                        tf_acquisition.acquire_haadf(
                            exposure=exposure,
                            resolution=resolution,
                            return_adorned_object=True,
                            return_pixel_size=True,
                        )
                    )

                    # Store image data
                    images.append(haadf_image)
                    image_data.append(haadf_data)

                    # Generate systematic filename
                    new_filename = output_dir / f"haadf_{grid_x:03d}_{grid_y:03d}.tiff"

                    # Move file to output directory with new name
                    if os.path.exists(haadf_filename):
                        os.rename(haadf_filename, str(new_filename))
                    filenames.append(str(new_filename))

                    print(f"  Image saved as: {new_filename.name}")

                    # Update metadata file
                    with open(metadata_file, "a") as f:
                        f.write(
                            f"{image_index}\t{grid_x}\t{grid_y}\t{actual_position.x * 1e6:.2f}\t{actual_position.y * 1e6:.2f}\t{new_filename.name}\n"
                        )

                except Exception as e:
                    print(
                        f"  Error acquiring image at position ({grid_x}, {grid_y}): {e}"
                    )
                    # Add placeholder for failed image
                    images.append(None)
                    image_data.append(None)
                    filenames.append(None)
                    positions.append((actual_position.x, actual_position.y))

                # Progress update
                if image_index % 10 == 0:
                    progress = (image_index / (ImagesX * ImagesY)) * 100
                    print(
                        f"  Progress: {progress:.1f}% ({image_index}/{ImagesX * ImagesY})"
                    )

    except KeyboardInterrupt:
        print("\nAcquisition interrupted by user!")
    except Exception as e:
        print(f"\nUnexpected error during acquisition: {e}")

    finally:
        # Always return to initial position
        print("Returning to initial stage position...")
        stage.absolute_move(InitialStagePosition)

    # Filter out failed acquisitions
    valid_images = [img for img in images if img is not None]
    valid_data = [data for data in image_data if data is not None]

    print(f"Successfully collected {len(valid_images)} out of {len(images)} images")

    if len(valid_data) > 0:
        # Create and save the stitched mosaic
        print("Creating stitched mosaic...")
        stitched_image = create_full_stage_mosaic(
            image_data, ImagesX, ImagesY, output_dir
        )

        # Save individual images as PNG for easy viewing
        print("Converting individual images to PNG...")
        convert_images_to_png(image_data, filenames, output_dir)

        # Create summary plot
        create_full_stage_summary_plot(valid_images, ImagesX, ImagesY, output_dir)

        # Create position map
        create_position_map(positions, ImagesX, ImagesY, output_dir, x_limits, y_limits)

        print(f"\nFull stage HAADF mosaic acquisition complete!")
        print(f"Results saved in: {output_dir}")
        print(f"Total images: {len(valid_images)}")
        print(f"Grid size: {ImagesX} × {ImagesY}")

        return {
            "images": valid_images,
            "image_data": valid_data,
            "positions": positions,
            "filenames": filenames,
            "stitched_image": stitched_image,
            "output_dir": output_dir,
            "pixel_size": pixel_size if len(valid_images) > 0 else None,
            "grid_size": (ImagesX, ImagesY),
            "stage_limits": {"x": x_limits, "y": y_limits},
        }
    else:
        print("No valid images acquired!")
        return None

In [4]:
def create_full_stage_mosaic(image_data, ImagesX, ImagesY, output_dir):
    """
    Create mosaic from full stage acquisition, handling missing images
    """
    if not any(data is not None for data in image_data):
        print("No valid images to stitch")
        return None

    # Get image dimensions from first valid image
    valid_data = [data for data in image_data if data is not None]
    if not valid_data:
        return None

    img_height, img_width = valid_data[0].shape

    # Create empty mosaic array
    mosaic_height = img_height * ImagesY
    mosaic_width = img_width * ImagesX
    mosaic = np.zeros((mosaic_height, mosaic_width), dtype=valid_data[0].dtype)

    # Place each image in the mosaic
    for grid_y in range(ImagesY):
        for grid_x in range(ImagesX):
            i = grid_x + grid_y * ImagesX

            if i < len(image_data) and image_data[i] is not None:
                # Calculate position in mosaic
                start_y = grid_y * img_height
                end_y = start_y + img_height
                start_x = grid_x * img_width
                end_x = start_x + img_width

                # Place image
                mosaic[start_y:end_y, start_x:end_x] = image_data[i]

    # Save mosaic as TIFF and PNG
    mosaic_tiff = output_dir / "full_stage_stitched_mosaic.tiff"
    mosaic_png = output_dir / "full_stage_stitched_mosaic.png"

    # Save as TIFF (preserving original data type)
    Image.fromarray(mosaic).save(str(mosaic_tiff))

    # Save as PNG (normalized for viewing)
    save_array_as_png(mosaic, mosaic_png)

    print(f"Full stage mosaic saved as: {mosaic_tiff.name} and {mosaic_png.name}")

    return mosaic

In [5]:
def convert_images_to_png(image_data, filenames, output_dir):
    """
    Convert all valid images to PNG format
    """
    png_dir = output_dir / "individual_pngs"
    png_dir.mkdir(exist_ok=True)

    converted = 0
    for i, (data, filename) in enumerate(zip(image_data, filenames)):
        if data is not None and filename is not None:
            png_path = png_dir / f"{Path(filename).stem}.png"
            save_array_as_png(data, png_path)
            converted += 1

    print(f"Converted {converted} images to PNG format in {png_dir.name}/")

In [6]:
def create_position_map(positions, ImagesX, ImagesY, output_dir, x_limits, y_limits):
    """
    Create a map showing the actual stage positions
    """
    fig, ax = plt.subplots(figsize=(12, 8))

    # Extract x and y coordinates
    x_coords = [pos[0] * 1e6 for pos in positions if pos is not None]  # Convert to μm
    y_coords = [pos[1] * 1e6 for pos in positions if pos is not None]

    # Plot positions
    scatter = ax.scatter(
        x_coords, y_coords, c=range(len(x_coords)), cmap="viridis", s=50, alpha=0.7
    )

    # Add colorbar
    plt.colorbar(scatter, ax=ax, label="Acquisition Order")

    # Set limits and labels
    ax.set_xlim(x_limits.min * 1e6, x_limits.max * 1e6)
    ax.set_ylim(y_limits.min * 1e6, y_limits.max * 1e6)
    ax.set_xlabel("Stage X Position (μm)")
    ax.set_ylabel("Stage Y Position (μm)")
    ax.set_title(f"Stage Position Map - {ImagesX}×{ImagesY} Grid")
    ax.grid(True, alpha=0.3)

    # Add stage limits rectangle
    from matplotlib.patches import Rectangle

    stage_rect = Rectangle(
        (x_limits.min * 1e6, y_limits.min * 1e6),
        (x_limits.max - x_limits.min) * 1e6,
        (y_limits.max - y_limits.min) * 1e6,
        linewidth=2,
        edgecolor="red",
        facecolor="none",
        label="Stage Limits",
    )
    ax.add_patch(stage_rect)
    ax.legend()

    plt.tight_layout()
    position_map_file = output_dir / "stage_position_map.png"
    plt.savefig(str(position_map_file), dpi=300, bbox_inches="tight")
    plt.close()

    print(f"Stage position map saved as: {position_map_file.name}")

In [7]:
def create_full_stage_summary_plot(images, ImagesX, ImagesY, output_dir):
    """
    Create summary plot for full stage acquisition (may be very large)
    """
    # If too many images, create a downsampled version
    max_display_images = 100

    if len(images) > max_display_images:
        # Sample images evenly
        step = len(images) // max_display_images
        sampled_images = images[::step]
        display_title = (
            f"Full Stage HAADF Mosaic - Sample of {len(sampled_images)} images"
        )

        # Calculate display grid
        display_cols = min(10, len(sampled_images))
        display_rows = math.ceil(len(sampled_images) / display_cols)
    else:
        sampled_images = images
        display_title = f"Full Stage HAADF Mosaic - All {len(images)} images"
        display_cols = min(ImagesX, 10)
        display_rows = math.ceil(len(images) / display_cols)

    fig, axes = plt.subplots(display_rows, display_cols, figsize=(20, 15))
    fig.suptitle(display_title, fontsize=16)

    # Ensure axes is 2D
    if display_rows == 1:
        axes = axes.reshape(1, -1)
    elif display_cols == 1:
        axes = axes.reshape(-1, 1)

    for i in range(display_rows * display_cols):
        row = i // display_cols
        col = i % display_cols
        ax = axes[row, col]

        if i < len(sampled_images) and sampled_images[i] is not None:
            ax.imshow(sampled_images[i].data, cmap="gray")
            ax.set_title(f"Img {i + 1}", fontsize=8)

        ax.axis("off")

    plt.tight_layout()
    summary_plot = output_dir / "full_stage_summary_plot.png"
    plt.savefig(str(summary_plot), dpi=200, bbox_inches="tight")
    plt.close()

    print(f"Summary plot saved as: {summary_plot.name}")

In [8]:
def save_array_as_png(array, filename):
    """
    Save numpy array as PNG with proper normalization
    """
    if array is None:
        return

    # Normalize to 0-255 range
    normalized = ((array - array.min()) / (array.max() - array.min()) * 255).astype(
        np.uint8
    )
    Image.fromarray(normalized).save(str(filename))

In [9]:
def run_full_stage_haadf_acquisition(config):
    """
    Main function to run the complete full-stage HAADF mosaic acquisition
    """
    # Initialize microscope (using your existing setup)
    from autoscript_tem_microscope_client import TemMicroscopeClient
    from stemOrchestrator.acquisition import TFacquisition

    microscope = TemMicroscopeClient()
    microscope.connect(config["ip"], port=config["port"])

    tf_acquisition = TFacquisition(microscope=microscope)

    # Ensure microscope is in proper state
    print("Setting up microscope for full-stage STEM imaging...")

    # Set STEM mode
    if tf_acquisition.microscope.optics.optical_mode != "STEM":
        tf_acquisition.microscope.optics.optical_mode = "STEM"
        print("Switched to STEM mode")

    # Unblank beam
    tf_acquisition.unblank_beam()
    print("Beam unblanked")

    # Retract camera if inserted
    if tf_acquisition.ceta_cam.insertion_state == "INSERTED":
        tf_acquisition.ceta_cam.retract()
        print("Camera retracted")

    # Open vacuum valves if needed
    tf_acquisition.open_vacuum_valves()

    # Run the full stage acquisition
    results = collect_full_stage_haadf_mosaic(tf_acquisition, config)

    if results:
        print("\n" + "=" * 60)
        print("FULL STAGE HAADF MOSAIC ACQUISITION COMPLETE!")
        print("=" * 60)
        print(f"Results saved in: {results['output_dir']}")
        print(f"Number of images collected: {len(results['images'])}")
        print(f"Grid size: {results['grid_size'][0]} × {results['grid_size'][1]}")
        print(
            f"Stage coverage: X = {results['stage_limits']['x'].min * 1e6:.1f} to {results['stage_limits']['x'].max * 1e6:.1f} μm"
        )
        print(
            f"                Y = {results['stage_limits']['y'].min * 1e6:.1f} to {results['stage_limits']['y'].max * 1e6:.1f} μm"
        )

    return results

## Configure Acquisition Parameters
Set up the configuration dictionary for full stage HAADF mosaic acquisition.

In [ ]:
# Define microscope connection info and acquisition parameters
ip = os.getenv("MICROSCOPE_IP")
port = os.getenv("MICROSCOPE_PORT")

if not ip or not port:
    secret_path = Path("../../config_secret.json")
    if secret_path.exists():
        with open(secret_path, "r") as f:
            secret = json.load(f)
            ip = ip or secret.get("ip_TF")
            port = port or secret.get("port_TF")

if not ip:
    ip = input("Enter microscope IP: ")
if not port:
    port = input("Enter microscope Port: ")
port = int(port)

config = {
    "ip": ip,
    "port": port,
    "haadf_exposure": 200e-9,  # 40 microseconds per pixel
    "haadf_resolution": 512,  # Can reduce to 256 for faster acquisition
    "overlap": 0.1,  # 10% overlap between images
    "out_path": ".",
}
print("Configuration ready.")

## Run Full Stage HAADF Mosaic Acquisition
This will cover the entire stage range in X and Y directions. **Warning:** This may take several hours depending on stage size!

In [ ]:
results = run_full_stage_haadf_acquisition(config)

## Display Stitched Mosaic
If acquisition was successful, display a downsampled version of the stitched mosaic.

In [ ]:
if results and results["stitched_image"] is not None:
    mosaic = results["stitched_image"]
    # Downsample for display if too large
    if mosaic.shape[0] > 2000 or mosaic.shape[1] > 2000:
        downsample_factor = max(mosaic.shape[0] // 2000, mosaic.shape[1] // 2000)
        mosaic_display = mosaic[::downsample_factor, ::downsample_factor]
    else:
        mosaic_display = mosaic
    plt.figure(figsize=(15, 10))
    plt.imshow(mosaic_display, cmap="gray")
    plt.title("Full Stage HAADF Mosaic (downsampled for display)")
    plt.axis("off")
    plt.tight_layout()
    plt.show()
else:
    print("No stitched mosaic to display.")